## AIS数据按每小时取平均，并且将所有数据合并到一个csv文件

In [1]:
import pandas as pd
import numpy as np
import movingpandas as mdp

/usr/local/python/3.12.1/lib/python3.12/site-packages/movingpandas/__init__.py:37: UserWarning: Missing optional dependencies. To use the trajectory smoother classes please install Stone Soup (see https://stonesoup.readthedocs.io/en/latest/#installation).
  warnings.warn(e.msg, UserWarning)


In [ ]:
# 读取船的AIS数据
shipnum = 1
df = pd.read_csv('processed_1.csv',index_col=0)
df['postime'] = pd.to_datetime(df['postime'],format='ISO8601')

# 读取noon report，并且筛选出年月日
df_fuel = pd.read_csv('../raw_data/mrv.csv')
# 时间转换成datetime对象
df_fuel['input_date'] = pd.to_datetime(df_fuel['input_date'],format='ISO8601')
# 先观察第一艘船
df_fuel = df_fuel[df_fuel['num'] == shipnum]
# 筛选出一直在跑的船，这边考虑daily_voyage_time_mrv大于等于24小时的轨迹
df_fuel = df_fuel[df_fuel['daily_voyage_time_mrv'] >= 24]
# 删除关键字段存在空值的行，daily_fuel_mrv
df_fuel = df_fuel.dropna(axis=0, subset = ['daily_fuel_mrv'])
# 序号重新排列
df_fuel = df_fuel.reset_index(drop=True)
# 将日期全部抽取出来
years = df_fuel['input_date'].dt.year
months = df_fuel['input_date'].dt.month
days = df_fuel['input_date'].dt.day



for i in range(1):
    for hours in range(1):
        # 同一个小时的所有点
        temp_hours = df[(df['postime'].dt.year ==years[i]) & (df['postime'].dt.month ==months[i]) & (df['postime'].dt.day ==days[i]) & (df['postime'].dt.hour == hours)]
        avg = temp_hours[['SOG','heading','draught', 'status', 'wind_val', 'wind_direction', 'wave_val', 'wave_direction', 'stream_val', 'stream_direction']].mean()

        # df_processed = pd.concat([df_processed, temp], ignore_index = True)



np.float64(13.920000000000002)